In [0]:
# ML_in_Finance-Deep-Learning-Interpretability
# Author: Matthew Dixon
# Version: 1.0 (08.09.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6 and Tensorflow 1.3.0
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Dixon M.F., I. Halperin and P. Bilokon, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 

# Overview
The purpose of this notebook is to illustrate a neural network interpretability method which is compatible with linear regression. In linear regression, provided the independent variables are scaled, one can view the regression coefficients as a measure of importance of the variables. Equivalently, the dependent variable can be differentiated w.r.t. the inputs to give the coefficient. Similarly, the derivatives of the network w.r.t. the inputs are a non-linear generalization of interpretability in linear regression. Moreover, we should expect the neural network gradients to approximate the linear regression coefficients when the data is generated by a linear regression model. Various simple experimental tests, corresponding to Section 3 of Chpt 5, are performed to illustrate the properties of network interpretability.

In [2]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import math
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.regularizers import l1,l2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score, train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
import statsmodels.api as sm
import statsmodels.stats.diagnostic as tds
from statsmodels.api import add_constant
from scipy import stats


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


# Simple Data Generation Process (DGP)


Let us generate data from the following linear regression model

$Y=X_1+X_2 + \epsilon, ~X_1, X_2, \epsilon \sim N(0,1)$

In [0]:
M = 5000 # Number of samples
np.random.seed(7) # set the seed for reproducebility of results
X = np.zeros(shape=(M,2))
X[:int(M/2),0]= np.random.randn(int(M/2))
X[:int(M/2),1]= np.random.randn(int(M/2))

# use antithetic sampling to reduce the bias in the mean
X[int(M/2):,0]= -X[:int(M/2),0]
X[int(M/2):,1]= -X[:int(M/2),1]


eps= np.zeros(shape=(M,1))
eps[:int(M/2)]= np.random.randn(int(M/2),1)
eps[int(M/2):]=-eps[:int(M/2)]
Y= 1.0*X[:,0] + 1.0*X[:,1] + eps.flatten()

In [26]:
np.mean(X,axis=0)

array([-2.51576537e-17, -5.48450174e-18])

# Use OLS to fit the model to the data
For a baseline, let us compare the neural network with OLS regression. You should observe that the intercept is close to zero and the other coefficients are close to one.

In [0]:
ols_results = sm.OLS(Y, sm.add_constant(X)).fit()

In [0]:
y_ols=ols_results.predict(sm.add_constant(X))

In [30]:
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     5249.
Date:                Tue, 12 May 2020   Prob (F-statistic):               0.00
Time:                        23:59:08   Log-Likelihood:                -7020.4
No. Observations:                5000   AIC:                         1.405e+04
Df Residuals:                    4997   BIC:                         1.407e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.939e-18      0.014  -4.98e-16      1.000      -0.027       0.027
x1             0.9858      0.014     70.409      0.000       0.958       1.013
x2             1.0190      0.014     72.695      0.000       0.992       1.047
==============================================================================
Omnibus:                        0.800   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.670   Jarque-Bera (JB):                0.750
Skew:                          -0.000   Prob(JB):                        0.687
Kurtosis:                       3.060   Cond. No.                         1.02
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Compare with a ffwd neural network with no hidden layers
Recall the feedforward network with no hidden layers is a linear regression model

In [0]:
def linear_NN0_model(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(1, input_dim=2, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [0]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

In [0]:
lm = KerasRegressor(build_fn=linear_NN0_model, epochs=40, batch_size=10, verbose=1, callbacks=[es])

In [34]:
lm.fit(X,Y)

Epoch 1/40
5000/5000 [==============================] - 1s 136us/step - loss: 2.4863 - mae: 1.2602 - mse: 2.4863
Epoch 2/40
5000/5000 [==============================] - 0s 85us/step - loss: 1.5447 - mae: 0.9995 - mse: 1.5447
Epoch 3/40
5000/5000 [==============================] - 0s 89us/step - loss: 1.1417 - mae: 0.8550 - mse: 1.1417
Epoch 4/40
5000/5000 [==============================] - 0s 85us/step - loss: 1.0090 - mae: 0.8026 - mse: 1.0090
Epoch 5/40
5000/5000 [==============================] - 0s 88us/step - loss: 0.9772 - mae: 0.7890 - mse: 0.9772
Epoch 6/40
5000/5000 [==============================] - 0s 87us/step - loss: 0.9719 - mae: 0.7869 - mse: 0.9719
Epoch 7/40
5000/5000 [==============================] - 0s 86us/step - loss: 0.9712 - mae: 0.7865 - mse: 0.9712
Epoch 8/40
5000/5000 [==============================] - 0s 85us/step - loss: 0.9713 - mae: 0.7865 - mse: 0.9713
Epoch 9/40
5000/5000 [==============================] - 0s 87us/step - loss: 0.9716 - mae: 0.7868 - mse

## Check that the weights are close to one
The weights should be close to unity. The bias term is the second entry and should be close to zero.

In [43]:
print("weights: " + str(lm.model.layers[0].get_weights()[0]))
print("bias: " + str(lm.model.layers[0].get_weights()[1]))

weights: [[0.9869313]
 [1.0236046]]
bias: [0.00017654]


# Compare with a FFW Neural Network with one hidden layer (unactivated)


In [0]:
n = 10 # number of hidden units

In [0]:
def linear_NN1_model(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(n, input_dim=2, kernel_initializer='normal')) 
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [0]:
lm = KerasRegressor(build_fn=linear_NN1_model, epochs=50, batch_size=10, verbose=1, callbacks=[es])

In [0]:
lm.fit(X,Y)

Epoch 1/50
5000/5000 [==============================] - 1s 144us/step - loss: 1.5925 - mean_absolute_error: 0.9781 - mean_squared_error: 1.5925
Epoch 2/50
5000/5000 [==============================] - 0s 92us/step - loss: 0.9818 - mean_absolute_error: 0.7901 - mean_squared_error: 0.9818
Epoch 3/50
5000/5000 [==============================] - 1s 105us/step - loss: 0.9818 - mean_absolute_error: 0.7912 - mean_squared_error: 0.9818
Epoch 4/50
5000/5000 [==============================] - 1s 112us/step - loss: 0.9822 - mean_absolute_error: 0.7914 - mean_squared_error: 0.9822
Epoch 5/50
5000/5000 [==============================] - 0s 95us/step - loss: 0.9818 - mean_absolute_error: 0.7919 - mean_squared_error: 0.9818
Epoch 6/50
5000/5000 [==============================] - 1s 114us/step - loss: 0.9822 - mean_absolute_error: 0.7906 - mean_squared_error: 0.9822
Epoch 7/50
5000/5000 [==============================] - 0s 92us/step - loss: 0.9814 - mean_absolute_error: 0.7904 - mean_squared_error: 0.

In [0]:
W1=lm.model.get_weights()[0]
b1=lm.model.get_weights()[1]
W2=lm.model.get_weights()[2]
b2=lm.model.get_weights()[3]
print(W1, W2)

[[-0.28151304  0.33660048 -0.32534724 -0.3117283   0.36416572  0.31704733
  -0.3224496   0.32124516  0.28360805  0.40207058]
 [-0.3099645   0.27803436 -0.37927535 -0.2829823   0.2626483   0.31276977
  -0.3437224   0.31792367  0.37649897  0.31318784]] [[-0.35752591]
 [ 0.33028835]
 [-0.28204554]
 [-0.31195363]
 [ 0.2602262 ]
 [ 0.344295  ]
 [-0.3088067 ]
 [ 0.30915156]
 [ 0.29915127]
 [ 0.27908695]]


## Check that the coefficients are close to one and the intercept is close to zero

In [0]:
beta_0=np.dot(np.transpose(W2), b1) + b2
beta_1=np.dot(np.transpose(W2), W1[0])
beta_2=np.dot(np.transpose(W2), W1[1])

In [0]:
print(beta_0,beta_1,beta_2)

[0.02593422] [1.0006967] [0.9784023]


# Compare with a FFW Neural Network with one hidden layer (tanh activated)

In [0]:
# number of hidden neurons
n=10

In [0]:
# with non-linear activation
def linear_NN1_model_act(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(n, input_dim=2, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [0]:
lm = KerasRegressor(build_fn=linear_NN1_model_act, epochs=100, batch_size=10, verbose=1, callbacks=[es])

In [0]:
lm.fit(X,Y)

Epoch 1/100
5000/5000 [==============================] - 1s 149us/step - loss: 1.6640 - mean_absolute_error: 1.0059 - mean_squared_error: 1.6640
Epoch 2/100
5000/5000 [==============================] - 0s 92us/step - loss: 1.0040 - mean_absolute_error: 0.8007 - mean_squared_error: 1.0040
Epoch 3/100
5000/5000 [==============================] - 0s 92us/step - loss: 1.0032 - mean_absolute_error: 0.8005 - mean_squared_error: 1.0032
Epoch 4/100
5000/5000 [==============================] - 0s 95us/step - loss: 0.9997 - mean_absolute_error: 0.7989 - mean_squared_error: 0.9997
Epoch 5/100
5000/5000 [==============================] - 0s 100us/step - loss: 0.9967 - mean_absolute_error: 0.7976 - mean_squared_error: 0.9967
Epoch 6/100
5000/5000 [==============================] - 0s 93us/step - loss: 0.9960 - mean_absolute_error: 0.7972 - mean_squared_error: 0.9960
Epoch 7/100
5000/5000 [==============================] - 0s 96us/step - loss: 0.9949 - mean_absolute_error: 0.7970 - mean_squared_erro

## Compute the Sensitivities

In [0]:
# Assume that the activation function is tanh
def sensitivities(lm, X):
    
    W1=lm.model.get_weights()[0]
    b1=lm.model.get_weights()[1]
    W2=lm.model.get_weights()[2]
    b2=lm.model.get_weights()[3]
    
    
    M = np.shape(X)[0]
    p = np.shape(X)[1]

    beta=np.array([0]*M*(p+1), dtype='float32').reshape(M,p+1)
    
    beta[:,0]= (np.dot(np.transpose(W2),np.tanh(b1)) + b2)[0] # intercept \beta_0= F_{W,b}(0)
    for i in range(M):
 
      Z1 = np.tanh(np.dot(np.transpose(W1),np.transpose(X[i,])) + b1)
      #Z1 = np.maximum(np.dot(np.transpose(W1),np.transpose(X[i,])) + b1,0) (ReLU)
      
      D = np.diag(1-Z1**2)
      #D = np.diag(np.sign(Z1))  (ReLU)
        
      for j in range(p):  
          beta[i,j+1]=np.dot(np.transpose(W2),np.dot(D,W1[j]))
            
    return(beta)

In [0]:
beta=sensitivities(lm, X)

## Check that the intercept is close to one and the coefficients are close to one

In [0]:
print(np.mean(beta, axis=0))

[-0.0288713   0.9822788   0.98537195]


In [0]:
print(np.std(beta, axis=0))

[1.8812773e-06 5.4207556e-02 5.4662943e-02]
